In [32]:

def TR(high, low, preClose):
    return max(abs(high-low), abs(high-preClose), abs(preClose-low))

def DM(high, preHigh, low, preLow):
    deltaHigh = high - preHigh
    deltaLow = preLow - low

    DMplus = None
    DMminus = None
    if (deltaHigh < 0 and deltaLow < 0) or deltaHigh == deltaLow:
        DMplus = 0
        DMminus = 0
    elif deltaHigh > deltaLow:
        DMplus = deltaHigh
        DMminus = 0
    elif deltaHigh < deltaLow:
        DMplus = 0
        DMminus = deltaLow
    else:
        raise 

    return DMplus, DMminus

def DIplus(preADMplus, preATR, DMplus, TR, n):
    return 100 * ((n-1)*preADMplus + DMplus) / ((n-1)*preATR + TR)

def DIminus(preADMminus, preATR, DMminus, TR, n):
    return 100 * (((n-1)*preADMminus + DMminus)) / ((n-1)*preATR + TR)

def _DX(DIplus, DIminus):
    return (DIplus - DIminus) / (DIplus + DIminus)

def DX(preADMplus, preADMminus, preATR, DMplus, DMminus, n,
      high, preHigh, low, preLow,
      preClose):
    DMplus, DMminus = DM(high, preHigh, low, preLow)
    TR_val = TR(high, low, preClose)
    DIplus_val = DIplus(preADMplus, preATR, DMplus, TR, n)
    DIminus_val = DIminus(preADMminus, preATR, DMminus, TR, n)
    
    DX_val = _DX(DIplus_val, DIminus_val)
    
    return DX_val
    


In [119]:
import numpy

def TR(inputs: dict):
    '''
    向量运算
    '''
    high = inputs['high']
    low = inputs['low']
    close = inputs['close']
    preClose = shift(close, 1, cval=np.nan)
    
    return np.amax(np.array([abs(high-low), abs(high-preClose), abs(preClose-low)]), axis=0)

def ATR(TR: numpy.ndarray, n):
    '''
    增量算法
    '''
    atr = np.full_like(tr, fill_value=np.nan)
    set_first_atr = False
    for i in range(len(tr)):
        if not set_first_atr:
            if not np.isnan(tr[i]):
                atr[i] = tr[i]
                set_first_atr = True
        else:
            atr[i] = ((n-1)/n)*atr[i-1] + tr[i]/n    
            
    return atr

def DM(inputs: dict):
    '''
    向量运算
    '''
    high = inputs['high']

    preHigh = shift(high, 1, cval=np.nan)
    low = inputs['low']
    preLow = shift(low, 1, cval=np.nan)

    deltaHigh = high - preHigh
    deltaLow = preLow - low

    DMplus = np.full_like(deltaHigh, fill_value=np.nan)
    DMminus = np.full_like(deltaHigh, fill_value=np.nan)

    condition1= np.logical_or(np.logical_and(deltaHigh < 0, deltaLow < 0), deltaHigh == deltaLow)
    DMplus[condition1] = 0
    DMminus[condition1] = 0
    condition2 = np.logical_and(deltaHigh > deltaLow, np.isnan(DMplus))
    DMplus[condition2] = deltaHigh[condition2]
    DMminus[condition2] = 0
    condition3 = np.logical_and(deltaHigh < deltaLow, np.isnan(DMplus))
    DMplus[condition3] = 0
    DMminus[condition3] = deltaLow[condition3]
    
    return DMplus, DMminus

def ADM(DMplus:numpy.ndarray, DMminus: numpy.ndarray, n) -> (numpy.ndarray, numpy.ndarray):
    '''
    增量算法
    '''
    admPlus = np.full_like(DMplus, fill_value=np.nan)
    admMinus = np.full_like(DMminus, fill_value=np.nan)
    set_first_adm = False
    assert len(DMplus) == len(DMminus)
    for i in range(len(DMplus)):
        if not set_first_adm:
            if not np.isnan(DMplus[i]):
                admPlus[i] = DMplus[i]
                admMinus[i] = DMminus[i]
                set_first_adm = True
        else:
            admPlus[i] = ((n-1)/n)*admPlus[i-1] + DMplus[i]/n    
            admMinus[i] = ((n-1)/n)*admMinus[i-1] + DMminus[i]/n

    return admPlus, admMinus
def DI(admPlus: numpy.ndarray, admMinus: numpy.ndarray, atr: numpy.ndarray) -> (numpy.ndarray, numpy.ndarray):
    '''
    向量运算
    '''
    DIplus = 100 * admPlus/atr
    DIminus = 100 * admMinus/atr
    
    return DIplus, DIminus

def DX_helper(DIplus: numpy.ndarray, DIminus: numpy.ndarray):
    '''
    向量运算
    '''
    return 100*abs(DIplus - DIminus)/(DIplus + DIminus)
def DX(inputs: dict, n: int):
    '''
    API
    '''
    np.set_printoptions(suppress=True)
    tr = TR(inputs)
    atr = ATR(tr, n)
    dmPlus, dmMinus = DM(inputs)
    admPlus, admMinus = ADM(dmPlus, dmMinus, n)
    DIplus, DIminus = DI(admPlus, admMinus, atr)

    
    return DX_helper(DIplus, DIminus)
    
def ADX_helper(dx: numpy.ndarray, n):
    '''
    增量算法
    '''
    adx = np.full_like(dx, fill_value=np.nan)
    set_first_adx = False
    for i in range(len(dx)):
        if not set_first_adx:
            if not np.isnan(dx[i]):
                adx[i] = dx[i]
                set_first_adx = True
        else:
            adx[i] = ((n-1)/n)*adx[i-1] + dx[i]/n    
            
    return adx    

def ADX(inputs: dict, n: int):
    '''
    API
    '''
    dx = DX(inputs, n)
    return ADX_helper(dx, n)

https://www.douban.com/note/270010228/

In [103]:
import numpy as np
import pandas as pd
from scipy.ndimage.interpolation import shift

inputs = {
    'open': np.random.random(100),
    'high': np.random.random(100),
    'low': np.random.random(100),
    'close': np.random.random(100),
    'volume': np.random.random(100)
}



    

In [116]:
ADX(inputs, 5)


/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:46: RuntimeWarning: invalid value encountered in less
/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: RuntimeWarning: invalid value encountered in greater
/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:52: RuntimeWarning: invalid value encountered in less


array([         nan, 100.        ,  84.7569065 ,  72.56243171,
        66.5583988 ,  61.75517247,  50.01882833,  46.74670677,
        40.71468571,  34.54644997,  30.04036895,  25.82854659,
        21.45414542,  17.95462449,  25.08206414,  21.63980992,
        24.67548447,  27.1040241 ,  32.26990472,  35.75100573,
        28.62859847,  30.49443042,  31.98709597,  33.18122842,
        39.78623967,  44.42520339,  48.13637436,  54.04583412,
        45.73414291,  43.55737161,  36.64113899,  30.27725905,
        25.1861551 ,  24.60718521,  22.69893201,  19.64319098,
        17.14653615,  18.05288549,  18.89528817,  18.05463821,
        17.38211825,  21.41670927,  25.01480182,  26.17525565,
        27.10361872,  28.04006791,  25.74004841,  23.90003282,
        27.80090443,  32.10042382,  35.54003933,  36.5964461 ,
        33.73904683,  33.11156841,  39.0791998 ,  43.85330491,
        40.06900481,  42.34256242,  35.0633874 ,  35.61974719,
        36.06483503,  40.57674739,  38.08721665,  40.39

In [121]:
import talib

talib.abstract.ADX(inputs, 5) - ADX(inputs, 5)

/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:46: RuntimeWarning: invalid value encountered in less
/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: RuntimeWarning: invalid value encountered in greater
/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:52: RuntimeWarning: invalid value encountered in less


array([         nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan, -12.77830392, -11.88552869,  -8.20863399,
        -6.97204915,  -5.98278127,  -4.31750013,  -3.09056941,
        -2.23758566,  -1.55519865,  -1.07516522,  -0.69352834,
        -0.48690059,  -0.31878678,  -0.18429574,  -0.0767029 ,
        -0.02517417,   0.01561033,   0.04823792,   0.05942089,
         0.06275819,   0.06101964,   0.04082983,   0.03944985,
         0.03834586,   0.02537076,   0.01537315,   0.01608715,
         0.00949851,   0.00467345,   0.00545043,   0.00307379,
         0.00117248,  -0.00004555,  -0.00099409,  -0.00167962,
        -0.00222804,  -0.00137561,  -0.00138469,  -0.00139195,
        -0.00131628,  -0.00123661,  -0.00117287,  -0.00110879,
        -0.00103005,  -0.00095381,  -0.00083656,  -0.00074275,
        -0.00064675,  -0.00055311,  -0.00046542,  -0.00038844,
        -0.00032686,  -0.00027262,  -0.0002266 ,  -0.00

In [110]:
def ADX(dx, n):
    adx = np.full_like(dx, fill_value=np.nan)
    set_first_adx = False
    for i in range(len(dx)):
        if not set_first_adx:
            if not np.isnan(dx[i]):
                adx[i] = dx[i]
                set_first_adx = True
        else:
            adx[i] = ((n-1)/n)*adx[i-1] + dx[i]/n    
            
    return adx    

In [111]:
ADX(dx,5)

array([         nan, 100.        ,  84.7569065 ,  72.56243171,
        66.5583988 ,  61.75517247,  50.01882833,  46.74670677,
        40.71468571,  34.54644997,  30.04036895,  25.82854659,
        21.45414542,  17.95462449,  25.08206414,  21.63980992,
        24.67548447,  27.1040241 ,  32.26990472,  35.75100573,
        28.62859847,  30.49443042,  31.98709597,  33.18122842,
        39.78623967,  44.42520339,  48.13637436,  54.04583412,
        45.73414291,  43.55737161,  36.64113899,  30.27725905,
        25.1861551 ,  24.60718521,  22.69893201,  19.64319098,
        17.14653615,  18.05288549,  18.89528817,  18.05463821,
        17.38211825,  21.41670927,  25.01480182,  26.17525565,
        27.10361872,  28.04006791,  25.74004841,  23.90003282,
        27.80090443,  32.10042382,  35.54003933,  36.5964461 ,
        33.73904683,  33.11156841,  39.0791998 ,  43.85330491,
        40.06900481,  42.34256242,  35.0633874 ,  35.61974719,
        36.06483503,  40.57674739,  38.08721665,  40.39

In [107]:
import talib
talib.abstract.DX(inputs, 5) - DX(inputs, 5)

/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:46: RuntimeWarning: invalid value encountered in less
/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: RuntimeWarning: invalid value encountered in greater
/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:52: RuntimeWarning: invalid value encountered in less


array([         nan,          nan,          nan,          nan,
                nan, -27.73575335,  17.56303831,  11.40924677,
        10.34810368,  -8.47795311,  -8.31442776,   6.4989448 ,
        -2.02570977,  -2.02570977,   2.34362443,   1.81715347,
         1.17434936,   1.17434936,   0.8449685 ,   0.8330192 ,
         0.33961042,   0.35366844,   0.35366844,   0.35366844,
         0.18094077,   0.17874831,   0.17874831,   0.10415274,
         0.07610738,   0.05406548,  -0.03992943,   0.03392991,
         0.03392991,  -0.02652961,  -0.02461732,   0.01894316,
        -0.01685607,  -0.01462676,   0.00855834,  -0.00643278,
        -0.00643278,  -0.00491768,  -0.00478825,  -0.00442173,
        -0.00442173,   0.0020341 ,  -0.00142098,  -0.00142098,
        -0.00101364,  -0.00091792,  -0.00091792,  -0.00085246,
        -0.00071508,  -0.00064888,  -0.00036752,  -0.00036752,
        -0.00026277,  -0.00017856,  -0.00011463,  -0.00008052,
        -0.00008052,  -0.00005567,  -0.00004252,  -0.00

In [58]:
def DX(DIplus: numpy.ndarray, DIminus: numpy.ndarray):
    
    return (DIplus - DIminus)/(DIplus + DIminus)

In [78]:
DMplus, DMminus = DM(inputs)

/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:46: RuntimeWarning: invalid value encountered in less
/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: RuntimeWarning: invalid value encountered in greater
/opt/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:52: RuntimeWarning: invalid value encountered in less


In [37]:
admPlus, admMinus = ADM(DMplus, DMminus, 5)

In [43]:
tr = TR(inputs)
atr = ATR(tr, 5)

In [50]:
DIplus, DIminus = DI(admPlus, admMinus, atr)

In [59]:
DX(DIplus, DIminus)

array([            nan, -1.00000000e+00, -1.00000000e+00, -1.83404498e+00,
       -2.01505784e+00,  2.73019198e+01,  4.15202525e-01,  6.18107959e-01,
        5.92185502e-01, -4.29190831e-01,  5.76178668e-01,  8.85702253e-01,
        8.27232520e-01,  6.51020819e-01, -1.66625959e-01, -3.21189852e-01,
        2.08994321e-01,  3.55924080e-01,  3.99569094e-01, -3.50129967e-01,
       -1.77783762e+00, -4.26704172e-01, -6.45649155e-01, -1.34944853e-01,
        8.08988261e-02,  1.01583128e-01, -2.16431787e-01, -4.33673638e-01,
        1.45835393e-01,  2.79952053e-02, -4.16424302e-01, -4.93321376e-01,
       -2.38316809e-01, -3.12514632e-01, -4.86858330e-01, -2.52946832e-01,
       -1.75432382e-01, -5.10631104e-01, -2.21796130e-01, -1.37425248e-01,
       -2.64763699e-01, -6.94487961e-02,  3.66856418e-01,  4.12430402e-01,
        3.39225150e-01, -2.25481084e-01, -6.32802718e-04,  2.71855498e-01,
        1.91708952e-03, -6.42234732e-01, -8.44188777e-01, -8.16888302e-01,
       -1.27711645e+00, -

In [ ]:
n = 5
atr = np.full_like(tr, fill_value=np.nan)
set_first_atr = False
for i in range(len(tr)):
    if not set_first_atr and not np.isnan(tr[i]):
        atr[i] = tr[i]
        set_first_atr = True
    else:
        atr[i] = ((n-1)/n)*atr[i-1] + tr[i]/n


In [ ]:
def ATR(TR, n):
    atr = np.full_like(tr, fill_value=np.nan)
    set_first_atr = False
    for i in range(len(tr)):
        if not set_first_atr and not np.isnan(tr[i]):
            atr[i] = tr[i]
            set_first_atr = True
        else:
            atr[i] = ((n-1)/n)*atr[i-1] + tr[i]/n    
            
    return atr

In [6]:
import scipy.ndimage
import numpy as np

scipy.ndimage.interpolation.shift(np.arange(8), 1, cval=np.nan)

array([-9223372036854775808,                    0,                    1,
                          2,                    3,                    4,
                          5,                    6])

In [12]:
i = 2

if i == 1:
    print(1)
elif i == 2:
    print(2)

2
